In [17]:
import requests
import os
import pandas as pd
import json
from tqdm import tqdm

index_dir='library_index.csv'


headers = {'x-api-key': 'api_key'}

match_api='https://api.semanticscholar.org/graph/v1/paper/search/match?query={title}'

index_csv=pd.read_csv(index_dir)
index_dict=index_csv.to_dict(orient='records')
index={}
for i in index_dict:
    index[i['acronym'].lower()]=i['title']
print(len(index))

297


## Check code base

In [ ]:
index=pd.read_csv('library_index.csv')
base=os.listdir('base')

PASS=True
for i in base:
    if i not in list(index['acronym']):
        print(i, 'not in index')
        PASS=False
    code=[i.split('.')[0] for i in os.listdir('base/'+i)]
    if f'{i}_edu' not in code:
        print(i, 'code not named correctly')
        PASS=False
if PASS:
    print('All checks passed')

## get paper id

In [19]:
papers={}
for i in tqdm(index):
    papers[i]=requests.get(match_api.format(title=index[i]), headers=headers).json()

100%|██████████| 300/300 [02:55<00:00,  1.71it/s]


In [20]:
index_csv['acronym']=index_csv['acronym'].str.lower()

In [23]:
unmatched=[]

for i in papers:
    try:
        id=papers[i]['data'][0]['paperId']
        index_csv.loc[index_csv['acronym']==i,'id']=id
    except:
        unmatched.append(i)
        print('Error:',i)

print(len(unmatched))

Error: ntk
Error: longnet
Error: feedbackmem
Error: srt
Error: kangpt
Error: s4pp
6


In [22]:

for i in tqdm(unmatched):
    papers[i]=requests.get(match_api.format(title=index[i])).json()

100%|██████████| 6/6 [00:02<00:00,  2.15it/s]


In [53]:
# index_csv.to_csv('library_index.csv',index=False)
for i in index_csv['acronym']:
    if index_csv.loc[index_csv['acronym']==i,'id'].isnull().values.any():
        print(i)

ntk
feedbackmem
srt
kangpt
s4pp


## Construct metadata & references

In [65]:
meta={}
meta_dir='library_meta.json'
if os.path.exists(meta_dir):
    with open(meta_dir,'r') as f:
        meta=json.load(f)

paper_detail='https://api.semanticscholar.org/graph/v1/paper/{paper_id}?fields=year,authors,tldr,venue,abstract,citationCount,influentialCitationCount,references,embedding.specter_v2,openAccessPdf'

for i in tqdm(index_csv.index):
    acronym=index_csv.loc[i,'acronym']
    title=index_csv.loc[i,'title']
    if 'detail' in meta.get(acronym,{}):
        if not 'message' in meta[acronym]['detail']:
            continue
    if not pd.isna(index_csv.loc[i,'id']):
        paper_id=index_csv.loc[i,'id']
        detail=requests.get(paper_detail.format(paper_id=paper_id)).json()#,headers=headers).json()
        if 'message' in detail:
            print('Error:',acronym,detail['message'])
            continue
        meta[acronym]={}
        meta[acronym]['title']=title
        meta[acronym]['id']=paper_id
        meta[acronym]['detail']=detail
    else:
        meta[acronym]={}
        meta[acronym]['title']=title
        meta[acronym]['id']=paper_id
        meta[acronym]['detail']={}

print(len(meta))

to_remove=[]
for i in meta:
    if i not in index:
        to_remove.append(i)
for i in to_remove:
    meta.pop(i)
    
print(len(meta))
with open(meta_dir,'w') as f:
    json.dump(meta,f)
        

references={}
ref_dir='library_ref.json'
if os.path.exists(ref_dir):
    with open(ref_dir,'r') as f:
        references=json.load(f)

references_detail='https://api.semanticscholar.org/graph/v1/paper/{paper_id}/references?fields=contextsWithIntent,intents,isInfluential,title,influentialCitationCount'

for i in tqdm(index_csv.index):
    acronym=index_csv.loc[i,'acronym']
    id=index_csv.loc[i,'id']
    if acronym in references:
        if 'message' not in references[acronym]:
            continue
    if pd.isna(id):
        references[acronym]=[]
    else:
        ret=requests.get(references_detail.format(paper_id=id)).json()#,headers=headers).json()
        if 'message' in ret:
            print('Error:',acronym,ret['message'])
            continue
        references[acronym]=ret
    
print(len(references))
to_remove=[]
for i in references:
    if i not in index:
        to_remove.append(i)
for i in to_remove:
    references.pop(i)
print(len(references))

with open(ref_dir,'w') as f:
    json.dump(references,f)


100%|██████████| 297/297 [00:00<00:00, 1041.46it/s]


297
297


100%|██████████| 297/297 [00:00<00:00, 86839.20it/s]


297
297


## Metadata to tree

In [4]:
index=pd.read_csv('./library_index.csv')
with open('./library_meta.json','r') as f:
    meta=json.load(f)
with open('./library_ref.json','r') as f:
    refs=json.load(f)
print(len(index),len(meta),len(refs))

NameError: name 'pd' is not defined

In [ ]:
meta['hydra']

In [ ]:
from dataclasses import dataclass, field, asdict
from typing import List, Dict


LIBRARY_DIR = './'

pjoin=os.path.join
pexists=os.path.exists

@dataclass
class NodeObject:
    acronym: str
    title: str
    seed_ids: List[str]

    def to_dict(self) -> Dict:
        return asdict(self)

    @classmethod
    def from_dict(cls, dict: Dict):
        return cls(**dict)
    
    @classmethod
    def load(cls, save_dir: str, acronym:str):
        with open(pjoin(save_dir,acronym+'.json'),'r') as f:
            return cls.from_dict(json.load(f))

    def save(self,save_dir: str):
        os.makedirs(save_dir, exist_ok=True)
        with open(pjoin(save_dir,self.acronym+'.json'),'w') as f:
            json.dump(self.to_dict(),f,indent=4)

    def to_desc(self) -> str:
        raise NotImplementedError

@dataclass
class LibraryReference(NodeObject):
    s2id: str = None
    abstract: str = None
    authors: List[str] = None
    venue: str = None
    year: int = None
    tldr: str = None
    # embedding: list
    citationCount: int = None
    influentialCitationCount: int = None
    code: str = None
    description: str = None
    url: str = None

    def __post_init__(self):
        code_dir=pjoin(LIBRARY_DIR,'base',self.acronym,self.acronym+'_edu.py')
        if pexists(code_dir):
            self.code=open(code_dir,'r').read()
        else:
            self.code=None

    @property
    def type(self) -> str:
        if self.code is not None:
            return 'ReferenceWithCode'
        else:
            return 'Reference'

    def to_desc(self) -> str:
        title=self.title.replace(':',' ')
        mdtext=f'# {title}'
        if self.s2id:
            mdtext+=f'\n* S2 ID {self.s2id} *'
        if self.authors:
            authors=', '.join(self.authors)
            mdtext+=f'\n* Authors: {authors} *'
        if self.tldr:
            tldr=self.tldr.replace(':',' ').replace(',',',\n')
            mdtext+=f'\n\n* TL;DR {tldr} *'
        if self.abstract:
            abstract=self.abstract.replace(':',' ').replace('.','.\n')
            mdtext+=f'\n\n## Abstract\n{abstract}'
        if self.venue:
            venue=self.venue.replace(':',' ')
            mdtext+=f'\n\n* Published at {venue} in {self.year} *'
            mdtext+=f'\n* Cited {self.citationCount} times *'
            mdtext+=f'\n* Impactful citations {self.influentialCitationCount} *'
        if self.description:
            description=self.description.replace(':',' ').replace('.','.\n')
            mdtext+=f'\n\n## Description\n{description}'
        return mdtext


manual_input={
    'srt': {
        'title': 'Self Reasoning Tokens',
        'authors': ['Felipe Sens Bonetto'],
        'year': 2024,
        'url': 'https://github.com/lucidrains/self-reasoning-tokens-pytorch',
        'description': "The project \"Reasoning Tokens\" by Felipe Sens Bonetto aims to enhance the reasoning abilities of language models like GPT by teaching them to plan ahead in a self-supervised way. The core idea is to introduce \"reasoning tokens,\" where for each token predicted, an additional token is generated that duplicates the input and doesn't receive a gradient from the next token but from future tokens. This approach encourages the model to pre-cache information useful for future predictions. Initial experiments showed a significant reduction in loss, indicating improved performance. The project plans to explore this method further, especially in fine-tuned instruction-following models, potentially replacing the need for step-by-step explanations during training. The ultimate goal is to create models that can reason internally, improving their performance and reducing the need for manually crafted training data.",
        'seed_ids': ['gpt3']
    },
    'ntk': {
        'title': 'NTK-Aware Scaled RoPE',
        'authors': ['bloc97'],
        'year': 2023,
        'url': 'https://www.reddit.com/r/LocalLLaMA/comments/14lz7j5/ntkaware_scaled_rope_allows_llama_models_to_have',
        'description': "The \"NTK-Aware Scaled RoPE\" project aims to extend the context size of LLaMA models beyond 8k tokens without fine-tuning and with minimal perplexity degradation. Traditional methods like RoPE interpolation often fail to distinguish between closely positioned tokens, leading to performance issues. By applying Neural Tangent Kernel (NTK) theory, this new method uses a nonlinear interpolation scheme that changes the RoPE's base rather than its scale, allowing for accurate distinction of token positions. This approach enables the LLaMA 7B model to handle longer contexts (up to 12k tokens) with minimal perplexity degradation, without fine-tuning. Initial tests show promising results, suggesting that further fine-tuning could enhance performance even more. The method provides a new way to extend the context window size efficiently, potentially benefiting tasks like long document summarization. The author encourages further experimentation and innovation in this area.",
        'seed_ids': ['roformer']
    },
    'feedbackmem': {
        'title': 'Addressing Some Limitations of Transformers with Feedback Memory',
        'authors': ['Angela Fan', 'Thibaut Lavril', 'Edouard Grave', 'Armand Joulin', 'Sainbayar Sukhbaatar'],
        'venue': 'arXiv',
        'year': 2020,
        'abstract': "Transformers have been successfully applied to sequential, auto-regressive tasks despite being feedforward networks. Unlike recurrent neural networks, Transformers use attention to capture temporal relations while processing input tokens in parallel. While this parallelization makes them computationally efficient, it restricts the model from fully exploiting the sequential nature of the input. The representation at a given layer can only access representations from lower layers, rather than the higher level representations already available. In this work, we propose the Feedback Transformer architecture that exposes all previous representations to all future representations, meaning the lowest representation of the current timestep is formed from the highest-level abstract representation of the past. We demonstrate on a variety of benchmarks in language modeling, machine translation, and reinforcement learning that the increased representation capacity can create small, shallow models with much stronger performance than comparable Transformers.",
        'tldr': 'Transformers have shortcomings - limited memory and limited state update - but Feedback Memory is a straightforward way to resolve these. ',
        'seed_ids': ['transformer','bert']
    },
    'kangpt': {
        'title': 'Generative Pre-trained Transformers (GPTs) using Kolmogorov-Arnold Networks (KANs) for language modeling',
        'authors': ['Aditya N Ganesh'],
        'year': 2024,
        'url': 'https://adityang.github.io/kan-gpt/',
        'description': "Kolmogorov-Arnold Networks (KANs) are promising alternatives of Multi-Layer Perceptrons (MLPs). KANs have strong mathematical foundations just like MLPs: MLPs are based on the universal approximation theorem, while KANs are based on Kolmogorov-Arnold representation theorem. KANs and MLPs are dual: KANs have activation functions on edges, while MLPs have activation functions on nodes. This simple change makes KANs better (sometimes much better!) than MLPs in terms of both model accuracy and interpretability. ",
        'seed_ids': ['gpt3','transformer']
    },
    's4pp': {
        'title': 'S4++: Elevating Long Sequence Modeling with State Memory Reply',
        'authors': ['Biqing Qi', 'Junqi Gao', 'Dong Li', 'Kaiyan Zhang', 'Jianxing Liu', 'Ligang Wu', 'Bowen Zhou'],
        'venue': 'ICLR 2024 Withdrawn Submission',
        'year': 2024,
        'url': 'https://openreview.net/forum?id=bdnw4qjfH9',
        'abstract': "Recently, state space models (SSMs) have shown significant performance advantages in modeling long sequences. However, in spite of their promising performance, there still exist limitations. 1. Non-Stable-States (NSS): Significant state variance discrepancies arise among discrete sampling steps, occasionally resulting in divergence. 2. Dependency Bias: The unidirectional state space dependency in SSM impedes the effective modeling of intricate dependencies. In this paper, we conduct theoretical analysis of SSM from the even-triggered control (ETC) theory perspective and first propose the presence of NSS Phenomenon. Our findings indicate that NSS primarily results from the sampling steps, and the integration of multi-state inputs into the current state significantly contributes to the mitigation of NSS. Building upon these theoretical analyses and findings, we propose a simple, yet effective, theoretically grounded State Memory Reply (SMR) mechanism that leverages learnable memories to incorporate multi-state information into the current state. This enables the precise modeling of finer state dependencies within the SSM, resulting in the introduction of S4+. Furthermore, we integrate the complex dependency bias into S4+ via interactive cross attentions mechanism, resulting in the development of S4++. Our extensive experiments in autoregressive language modeling and benchmarking against the Long Range Arena demonstrate superior performance in most post-processing tasks.",
        'seed_ids': ['s4']
    }
}



for acronym in refs:
    if refs[acronym]==[]:
        obj=manual_input[acronym]
        obj['acronym']=acronym
        proj=LibraryReference.from_dict(obj)
        proj.save('./tree')
        continue   
    refdata=refs[acronym]['data']
    seed_ids=[]
    for ref in refdata:
        if 'methodology' in ref['intents']:
            if ref['citedPaper']['paperId'] in index['id'].values:
                ref_acronym=index.loc[index['id']==ref['citedPaper']['paperId'],'acronym'].values[0]
                seed_ids.append(ref_acronym)
    title=meta[acronym]['title']
    s2id=meta[acronym]['id']
    abstract=meta[acronym]['detail']['abstract']
    authors=[author['name'] for author in meta[acronym]['detail']['authors']]
    if abstract is None:
        abstract='N/A'
    venue=meta[acronym]['detail']['venue']
    if venue is None:
        venue='arXiv'
    year=meta[acronym]['detail']['year']
    if year is None:
        year='N/A'
    tldr=meta[acronym]['detail']['tldr']
    if tldr is None:
        tldr='N/A'
    else:
        tldr=tldr['text']
        if tldr is None: tldr='N/A'
    embedding=meta[acronym]['detail']['embedding']
    if embedding is None:
        embedding=[]
    else:
        embedding=embedding['vector']
    citationCount=meta[acronym]['detail']['citationCount']
    influentialCitationCount=meta[acronym]['detail']['influentialCitationCount']
    paper=LibraryReference(title=title,acronym=acronym,seed_ids=seed_ids,s2id=s2id,abstract=abstract,authors=authors,venue=venue,year=year,tldr=tldr,citationCount=citationCount,influentialCitationCount=influentialCitationCount)
    paper.save('./tree')

print(len(os.listdir('./tree')))
# for i in os.listdir('./tree'):
#     if i.split('.')[0] not in refs:
#         print(i)
#         os.remove(pjoin('./tree',i))

297


## Build 1 hop impactful cites

In [2]:
index=pd.read_csv('./library_index.csv')
with open('./library_meta.json','r') as f:
    meta=json.load(f)
print(len(index),len(meta))
dir_1hop='./tree_ext'
get_cite='https://api.semanticscholar.org/graph/v1/paper/{paper_id}/citations?fields=intents,contextsWithIntent,isInfluential,title&offset={offset}&limit=1000'
paper_detail='https://api.semanticscholar.org/graph/v1/paper/{paper_id}?fields=year,authors,tldr,venue,abstract,citationCount,influentialCitationCount,references,embedding.specter_v2,openAccessPdf'

297 297


In [3]:
import time

cite_1hop_dir=dir_1hop+'/cite_1hop.json'
# detail_1hop_dir=dir_1hop+'/detail_1hop.json'
os.makedirs(dir_1hop,exist_ok=True)

cite_1hop={}
if os.path.exists(cite_1hop_dir):
    with open(cite_1hop_dir,'r') as f:
        cite_1hop=json.load(f)

for acronym in tqdm(index['acronym']):
    id=index.loc[index['acronym']==acronym,'id'].values[0]
    if pd.isna(id):
        cite_1hop[acronym]=[]
        continue
    citecount=meta[acronym]['detail']['citationCount']
    if acronym in cite_1hop:
        if 'message' not in cite_1hop[acronym]:
            # print('Already done:',acronym,len(cite_1hop[acronym]),citecount)
            continue
    cite_1hop[acronym]=[]
    maxoffset=min(citecount,9001)
    for offset in range(0,maxoffset,1000):
        if offset+1000>=10000:
            offset=8999
        print(acronym,offset,offset+1000)
        cites=requests.get(get_cite.format(paper_id=id,offset=offset)).json()#,headers=headers).json()
        if 'message' in cites:
            print('Error:',id,cites['message'])
            continue
        if 'data' not in cites:
            print('Error:',cites)
            raise
        for c in cites['data']:
            paperid=c['citingPaper']['paperId']
            if paperid in cite_1hop[acronym]:
                continue
            if 'methodology' in c['intents'] and c['isInfluential']:
                cite_1hop[acronym].append(paperid)
        time.sleep(0.1)
    print('Done',acronym,len(cite_1hop[acronym]),citecount)
    with open(cite_1hop_dir,'w') as f:
        json.dump(cite_1hop,f)

# to_remove=[]
# for i in cite_1hop:
#     if i not in index['acronym'].values:
#         to_remove.append(i)
# for i in to_remove:
#     cite_1hop.pop(i)
# with open(cite_1hop_dir,'w') as f:
#     json.dump(cite_1hop,f)

100%|██████████| 297/297 [00:00<00:00, 5183.07it/s]


### Load paper details for 1 hop

In [21]:
from tqdm import tqdm
import requests
import os
import json
import time

with open(cite_1hop_dir,'r') as f:
    cite_1hop=json.load(f)

index_1hop={}
for i in cite_1hop:
    for j in cite_1hop[i]:
        index_1hop[j]={}

paper_detail='https://api.semanticscholar.org/graph/v1/paper/{paper_id}?fields=title,year,authors,tldr,venue,abstract,citationCount,influentialCitationCount,references,embedding.specter_v2,openAccessPdf'

dir_meta='./tree_ext/meta'
os.makedirs(dir_meta,exist_ok=True)
for c in tqdm(index_1hop):
    if c is None: continue
    detail_1hop_dir=dir_meta+'/'+c+'.json'
    if os.path.exists(detail_1hop_dir):
        detail=json.load(open(detail_1hop_dir,'r'))
        if 'title' in detail:
            continue
    detail=requests.get(paper_detail.format(paper_id=c),headers=headers).json()
    time.sleep(0.2)
    if 'message' in detail:
        print('Error:',c,detail['message'])
        continue
    with open(detail_1hop_dir,'w') as f:
        json.dump(detail,f)

100%|██████████| 6856/6856 [00:02<00:00, 2980.03it/s]


## 1 hoc tree

In [23]:
import json,os

detail_1hop_dir='./tree_ext/meta'
cite_1hop_dir='./tree_ext/cite_1hop.json' 

with open(cite_1hop_dir,'r') as f:
    cite_1hop=json.load(f)

details={}
for i in os.listdir(detail_1hop_dir):
    with open(os.path.join(detail_1hop_dir,i),'r') as f:
        details[i.split('.')[0]]=json.load(f)
print(len(details))

6855


In [24]:
details['4064ebec539718b2d77607df8d86c6d436def83e'].keys()

dict_keys(['paperId', 'title', 'abstract', 'venue', 'year', 'citationCount', 'influentialCitationCount', 'openAccessPdf', 'tldr', 'embedding', 'authors', 'references'])

In [25]:
from dataclasses import dataclass, field, asdict
from typing import List, Dict


LIBRARY_DIR = './'

pjoin=os.path.join
pexists=os.path.exists

@dataclass
class NodeObject:
    acronym: str
    title: str
    seed_ids: List[str]

    def to_dict(self) -> Dict:
        return asdict(self)

    @classmethod
    def from_dict(cls, dict: Dict):
        return cls(**dict)
    
    @classmethod
    def load(cls, save_dir: str, acronym:str):
        with open(pjoin(save_dir,acronym+'.json'),'r') as f:
            return cls.from_dict(json.load(f))

    def save(self,save_dir: str):
        os.makedirs(save_dir, exist_ok=True)
        with open(pjoin(save_dir,self.acronym+'.json'),'w') as f:
            json.dump(self.to_dict(),f,indent=4)

    def to_desc(self) -> str:
        raise NotImplementedError

@dataclass
class LibraryReference(NodeObject):
    s2id: str = None
    abstract: str = None
    authors: List[str] = None
    venue: str = None
    year: int = None
    tldr: str = None
    # embedding: list
    citationCount: int = None
    influentialCitationCount: int = None
    code: str = None
    description: str = None
    url: str = None

    def __post_init__(self):
        code_dir=pjoin(LIBRARY_DIR,'base',self.acronym,self.acronym+'_edu.py')
        if pexists(code_dir):
            self.code=open(code_dir,'r').read()
        else:
            self.code=None

    @property
    def type(self) -> str:
        if self.code is not None:
            return 'ReferenceWithCode'
        else:
            return 'Reference'

    def to_desc(self) -> str:
        title=self.title.replace(':',' ')
        mdtext=f'# {title}'
        if self.s2id:
            mdtext+=f'\n* S2 ID {self.s2id} *'
        if self.authors:
            authors=', '.join(self.authors)
            mdtext+=f'\n* Authors: {authors} *'
        if self.tldr:
            tldr=self.tldr.replace(':',' ').replace(',',',\n')
            mdtext+=f'\n\n* TL;DR {tldr} *'
        if self.abstract:
            abstract=self.abstract.replace(':',' ').replace('.','.\n')
            mdtext+=f'\n\n## Abstract\n{abstract}'
        if self.venue:
            venue=self.venue.replace(':',' ')
            mdtext+=f'\n\n* Published at {venue} in {self.year} *'
            mdtext+=f'\n* Cited {self.citationCount} times *'
            mdtext+=f'\n* Impactful citations {self.influentialCitationCount} *'
        if self.description:
            description=self.description.replace(':',' ').replace('.','.\n')
            mdtext+=f'\n\n## Description\n{description}'
        return mdtext

class LibraryReference1hop(LibraryReference):

    def type(self):
        if self.code is not None:
            return 'Reference1hopWithCode'
        else:
            return 'Reference1hop'


for id in details:
    if os.path.exists(f'./tree_ext/1hop/{id}.json'):
        continue
    seed_ids=[]
    detail=details[id]
    references=[ref['paperId'] for ref in detail['references']]
    for i in cite_1hop:
        if id in cite_1hop[i]:
            seed_ids.append(i)
    for i in references:
        if i in details:
            seed_ids.append(i)
    s2id=id
    acronym = id
    title = detail['title']
    abstract = detail['abstract']
    authors = [author['name'] for author in detail['authors']]
    venue = detail['venue']
    year = detail['year']
    tldr = detail['tldr']
    citationCount = detail['citationCount']
    influentialCitationCount = detail['influentialCitationCount']
    seed_ids: List[str]
    paper=LibraryReference1hop(title=title,acronym=acronym,seed_ids=seed_ids,s2id=s2id,abstract=abstract,authors=authors,venue=venue,year=year,tldr=tldr,citationCount=citationCount,influentialCitationCount=influentialCitationCount)
    paper.save('./tree_ext/1hop')
    